# 🪟 Window Functions: Advanced Analytical Operations

**Time to complete:** 40 minutes  
**Difficulty:** Advanced  
**Prerequisites:** DataFrame basics, aggregations

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **Window functions** - Row-based analytical operations
- ✅ **Ranking functions** - rank(), dense_rank(), row_number()
- ✅ **Aggregate over windows** - running totals, moving averages
- ✅ **Lag/Lead functions** - Access previous/next rows
- ✅ **Complex window specifications** - PARTITION BY, ORDER BY, frame clauses
- ✅ **Performance optimization** - Efficient window operations

**Window functions are essential for advanced analytics and time-series analysis!**

---

## 🔍 Understanding Window Functions

**Window functions** perform calculations across sets of rows that are related to the current row. Unlike aggregations that group rows, window functions maintain all rows while adding analytical columns.

### Key Concepts:
- **Window**: Set of rows to analyze
- **Partition**: How to group rows (like GROUP BY)
- **Order**: How to sort rows within partitions
- **Frame**: Which rows within the partition to consider

### Window Function Syntax:
```sql
window_function() OVER (
    PARTITION BY column1, column2     -- Group rows
    ORDER BY column3                  -- Sort within groups  
    ROWS BETWEEN start AND end        -- Define frame
)
```

**Window functions are computed after aggregations but before final ORDER BY.**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, rank, dense_rank, lag, lead
from pyspark.sql.functions import sum, avg, min, max, count
from pyspark.sql.window import Window
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("Window_Functions") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark ready - Version: {spark.version}")

# Create comprehensive sales data with time component
sales_data = [
    ("Alice", "North", "Electronics", 1200, "2023-01-15"),
    ("Bob", "South", "Electronics", 800, "2023-01-16"),
    ("Alice", "North", "Clothing", 300, "2023-01-17"),
    ("Charlie", "East", "Electronics", 1500, "2023-01-18"),
    ("Alice", "North", "Electronics", 900, "2023-01-19"),
    ("Bob", "South", "Clothing", 450, "2023-01-20"),
    ("Diana", "West", "Electronics", 1300, "2023-01-21"),
    ("Charlie", "East", "Clothing", 600, "2023-01-22"),
    ("Alice", "North", "Books", 150, "2023-01-23"),
    ("Bob", "South", "Books", 200, "2023-01-24"),
    # February data
    ("Alice", "North", "Electronics", 1400, "2023-02-01"),
    ("Charlie", "East", "Electronics", 1600, "2023-02-02"),
    ("Bob", "South", "Electronics", 1100, "2023-02-03"),
    ("Diana", "West", "Clothing", 800, "2023-02-04"),
    ("Alice", "North", "Clothing", 500, "2023-02-05")
]

sales_df = spark.createDataFrame(
    sales_data, 
    ["salesperson", "region", "category", "amount", "date"]
)

print("📊 Sales Dataset:")
sales_df.orderBy("date").show()
print(f"Total records: {sales_df.count()}")

## 🏆 Ranking Functions

### Row Number, Rank, and Dense Rank

In [ ]:
# Ranking functions
print("🏆 RANKING FUNCTIONS")
print("=" * 50)

# Define window specifications
by_region_amount = Window.partitionBy("region").orderBy(col("amount").desc())
by_salesperson_date = Window.partitionBy("salesperson").orderBy("date")

# Apply ranking functions
ranked_df = sales_df.withColumn(
    "row_number", row_number().over(by_region_amount)
).withColumn(
    "rank", rank().over(by_region_amount)
).withColumn(
    "dense_rank", dense_rank().over(by_region_amount)
).withColumn(
    "sales_sequence", row_number().over(by_salesperson_date)
)

print("Ranking by region and amount (descending):")
ranked_df.select(
    "region", "salesperson", "amount", "row_number", "rank", "dense_rank"
).orderBy("region", col("amount").desc()).show()

print("\nSales sequence by salesperson and date:")
ranked_df.select(
    "salesperson", "date", "amount", "sales_sequence"
).orderBy("salesperson", "date").show()

### Understanding Ranking Differences

```python
# Sample data: [100, 100, 90, 80]
# row_number(): [1, 2, 3, 4]      # Always unique
# rank():       [1, 1, 3, 4]      # Ties get same rank, skip next
# dense_rank(): [1, 1, 2, 3]      # Ties get same rank, no skipping
```

### Top N Analysis

In [ ]:
# Top N analysis
print("🎯 TOP N ANALYSIS")
print("=" * 50)

# Top 2 salespeople by region
top_by_region = sales_df.withColumn(
    "rank_in_region", rank().over(
        Window.partitionBy("region").orderBy(col("amount").desc())
    )
).filter(col("rank_in_region") <= 2)

print("Top 2 sales by region:")
top_by_region.select(
    "region", "salesperson", "amount", "rank_in_region"
).orderBy("region", "rank_in_region").show()

# Top performers overall
overall_ranking = sales_df.withColumn(
    "overall_rank", rank().over(
        Window.orderBy(col("amount").desc())
    )
).filter(col("overall_rank") <= 5)

print("\nTop 5 sales overall:")
overall_ranking.select(
    "salesperson", "region", "category", "amount", "overall_rank"
).orderBy("overall_rank").show()

## 📈 Running Totals and Moving Aggregates

### Running Totals

In [ ]:
# Running totals
print("📈 RUNNING TOTALS")
print("=" * 50)

# Running total by salesperson over time
running_total_window = Window.partitionBy("salesperson").orderBy("date")

running_df = sales_df.withColumn(
    "running_total", sum("amount").over(running_total_window)
).withColumn(
    "running_count", count("*").over(running_total_window)
).withColumn(
    "running_avg", avg("amount").over(running_total_window)
)

print("Running totals by salesperson:")
running_df.select(
    "salesperson", "date", "amount", "running_total", "running_count", "running_avg"
).orderBy("salesperson", "date").show()

# Cumulative percentage
total_by_person = sales_df.groupBy("salesperson").agg(sum("amount").alias("total_amount"))
percent_window = Window.orderBy(col("total_amount").desc())

percentile_df = total_by_person.withColumn(
    "cumulative_amount", sum("total_amount").over(percent_window)
).withColumn(
    "total_overall", sum("total_amount").over())
).withColumn(
    "cumulative_percent", col("cumulative_amount") / col("total_overall") * 100
)

print("\nCumulative percentage of total sales:")
percentile_df.select(
    "salesperson", "total_amount", "cumulative_amount", "cumulative_percent"
).orderBy(col("total_amount").desc()).show()

### Moving Averages and Rolling Windows

In [ ]:
# Moving averages
print("📊 MOVING AVERAGES")
print("=" * 50)

# Moving average over last 3 sales per salesperson
moving_window = Window.partitionBy("salesperson").orderBy("date").rowsBetween(-2, 0)

moving_df = sales_df.withColumn(
    "moving_avg_3", avg("amount").over(moving_window)
).withColumn(
    "moving_sum_3", sum("amount").over(moving_window)
).withColumn(
    "moving_count", count("*").over(moving_window)
)

print("Moving averages (3-sale window):")
moving_df.select(
    "salesperson", "date", "amount", "moving_count", "moving_sum_3", "moving_avg_3"
).orderBy("salesperson", "date").show()

# Unbounded preceding (all previous rows)
unbounded_window = Window.partitionBy("salesperson").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

unbounded_df = sales_df.withColumn(
    "cumulative_avg", avg("amount").over(unbounded_window)
).withColumn(
    "cumulative_max", max("amount").over(unbounded_window)
)

print("\nCumulative statistics (unbounded preceding):")
unbounded_df.select(
    "salesperson", "date", "amount", "cumulative_avg", "cumulative_max"
).orderBy("salesperson", "date").show()

## 🔄 Lag and Lead Functions

### Accessing Previous and Next Rows

In [ ]:
# Lag and Lead functions
print("🔄 LAG AND LEAD FUNCTIONS")
print("=" * 50)

# Compare current sale with previous sales
lag_lead_df = sales_df.withColumn(
    "prev_sale", lag("amount", 1).over(
        Window.partitionBy("salesperson").orderBy("date")
    )
).withColumn(
    "next_sale", lead("amount", 1).over(
        Window.partitionBy("salesperson").orderBy("date")
    )
).withColumn(
    "sale_change", col("amount") - col("prev_sale")
).withColumn(
    "sale_trend", 
        when(col("amount") > col("prev_sale"), "↑ Increasing")
        .when(col("amount") < col("prev_sale"), "↓ Decreasing")
        .otherwise("→ Same")
)

print("Sale comparisons with previous/next:")
lag_lead_df.select(
    "salesperson", "date", "amount", "prev_sale", "next_sale", "sale_change", "sale_trend"
).orderBy("salesperson", "date").show()

# Performance analysis with lag
performance_df = sales_df.withColumn(
    "prev_amount", lag("amount", 1).over(
        Window.partitionBy("salesperson").orderBy("date")
    )
).withColumn(
    "improvement", 
        when(col("prev_amount").isNull(), "First Sale")
        .when(col("amount") > col("prev_amount"), "Improved")
        .when(col("amount") < col("prev_amount"), "Declined")
        .otherwise("Maintained")
)

print("\nPerformance analysis:")
performance_df.select(
    "salesperson", "date", "amount", "prev_amount", "improvement"
).orderBy("salesperson", "date").show()

## 🎛️ Advanced Window Specifications

### Frame Specifications

In [ ]:
# Advanced window specifications
print("🎛️ ADVANCED WINDOW SPECIFICATIONS")
print("=" * 50)

# Different frame types

# 1. ROWS frame (physical rows)
rows_frame = Window.partitionBy("region").orderBy("date").rowsBetween(-1, 1)

# 2. RANGE frame (logical range)
range_frame = Window.partitionBy("region").orderBy("amount").rangeBetween(-500, 500)

# Apply different frames
advanced_df = sales_df.withColumn(
    "rows_avg_3", avg("amount").over(rows_frame)
).withColumn(
    "range_count", count("*").over(range_frame)
)

print("Different window frame types:")
print("ROWS frame: Physical rows around current row")
print("RANGE frame: Logical range based on ORDER BY column\n")

advanced_df.select(
    "region", "date", "amount", "rows_avg_3", "range_count"
).orderBy("region", "date").show()

# Complex window with multiple functions
complex_window = Window.partitionBy("salesperson").orderBy("date")

complex_df = sales_df.withColumn(
    "sale_rank", row_number().over(complex_window)
).withColumn(
    "running_total", sum("amount").over(
        complex_window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
    )
).withColumn(
    "future_sales", sum("amount").over(
        complex_window.rowsBetween(1, 2)
    )
).withColumn(
    "sales_trend", 
        lag("amount", 1).over(complex_window) - lag("amount", 2).over(complex_window)
)

print("Complex multi-function windows:")
complex_df.select(
    "salesperson", "date", "amount", "sale_rank", "running_total", "future_sales", "sales_trend"
).orderBy("salesperson", "date").show()

## 📊 Business Analytics Examples

### Customer Segmentation and RFM Analysis

In [ ]:
# Business analytics
print("📊 BUSINESS ANALYTICS EXAMPLES")
print("=" * 50)

# Calculate salesperson metrics
salesperson_metrics = sales_df.groupBy("salesperson").agg(
    sum("amount").alias("total_sales"),
    count("*").alias("total_transactions"),
    avg("amount").alias("avg_sale"),
    max("amount").alias("best_sale"),
    countDistinct("category").alias("categories_sold"),
    countDistinct("region").alias("regions_active")
)

# Add performance ranking
rank_window = Window.orderBy(col("total_sales").desc())

performance_ranked = salesperson_metrics.withColumn(
    "sales_rank", rank().over(rank_window)
).withColumn(
    "sales_percentile", F.percentile_approx("total_sales", 0.5).over()
).withColumn(
    "performance_tier",
        when(col("total_sales") > col("sales_percentile") * 1.5, "Top Performer")
        .when(col("total_sales") > col("sales_percentile"), "Good Performer")
        .otherwise("Needs Improvement")
)

print("Salesperson Performance Ranking:")
performance_ranked.select(
    "sales_rank", "salesperson", "total_sales", "total_transactions", 
    "avg_sale", "performance_tier"
).orderBy("sales_rank").show()

# Monthly trends with window functions
monthly_window = Window.orderBy("date")

monthly_analysis = sales_df.withColumn(
    "month", F.date_format("date", "yyyy-MM")
).groupBy("month").agg(
    sum("amount").alias("monthly_sales"),
    count("*").alias("transaction_count"),
    countDistinct("salesperson").alias("active_sellers")
).withColumn(
    "prev_month_sales", lag("monthly_sales", 1).over(monthly_window)
).withColumn(
    "sales_growth", 
        when(col("prev_month_sales").isNotNull(), 
             (col("monthly_sales") - col("prev_month_sales")) / col("prev_month_sales") * 100
        ).otherwise(0)
).orderBy("month")

print("\nMonthly Sales Trends:")
monthly_analysis.select(
    "month", "monthly_sales", "transaction_count", "active_sellers", "sales_growth"
).show()

## ⚡ Performance Considerations

### Optimizing Window Functions

In [ ]:
# Performance optimization
print("⚡ WINDOW FUNCTION PERFORMANCE")
print("=" * 50)

# Create larger dataset for performance testing
large_data = [
    (f"salesperson_{i%100}", f"region_{(i%5)+1}", 100 + (i % 900), f"2023-{(i%12)+1:02d}-{i%28+1:02d}")
    for i in range(10000)
]

large_df = spark.createDataFrame(large_data, 
    ["salesperson", "region", "amount", "date"]
)

print(f"Large dataset: {large_df.count():,} records")

# Test different window strategies
import time

# Strategy 1: Multiple separate window operations
start_time = time.time()
window1 = Window.partitionBy("salesperson").orderBy("date")

result1 = large_df.withColumn("rank", rank().over(window1)) \
    .withColumn("running_total", sum("amount").over(window1)) \
    .filter(col("rank") <= 3)

strategy1_time = time.time() - start_time
print(f"Strategy 1 (multiple windows): {strategy1_time:.3f} seconds")

# Strategy 2: Pre-filter then window
start_time = time.time()
filtered_df = large_df.filter(col("amount") > 500)  # Reduce data first
window2 = Window.partitionBy("salesperson").orderBy("date")

result2 = filtered_df.withColumn("rank", rank().over(window2)) \
    .withColumn("running_total", sum("amount").over(window2)) \
    .filter(col("rank") <= 3)

strategy2_time = time.time() - start_time
print(f"Strategy 2 (pre-filter): {strategy2_time:.3f} seconds")

print(f"\nPre-filtering improved performance by {strategy1_time/strategy2_time:.1f}x")

# Best practices summary
print("\n🚀 WINDOW FUNCTION BEST PRACTICES:")
print("1. Filter data before applying windows")
print("2. Use appropriate partition keys")
print("3. Minimize window frame sizes")
print("4. Cache intermediate results if reused")
print("5. Consider approximate functions for large data")
print("6. Monitor Spark UI for window operation performance")

## 🚨 Common Mistakes and Debugging

In [ ]:
# Common mistakes
print("🚨 COMMON WINDOW FUNCTION MISTAKES")
print("=" * 50)

# Mistake 1: Forgetting ORDER BY in ranking functions
print("❌ Mistake: No ORDER BY in ranking")
bad_rank = sales_df.withColumn("bad_rank", rank().over(Window.partitionBy("region")))
print("Rank without ORDER BY (unpredictable results):")
bad_rank.select("region", "amount", "bad_rank").show(5)

print("\n✅ Correct: Include ORDER BY")
good_rank = sales_df.withColumn("good_rank", rank().over(
    Window.partitionBy("region").orderBy(col("amount").desc())
))
print("Rank with ORDER BY (correct results):")
good_rank.select("region", "amount", "good_rank").show(5)

# Mistake 2: Using wrong frame type
print("\n❌ Mistake: Using ROWS when RANGE is needed")
rows_frame = Window.partitionBy("region").orderBy("amount").rowsBetween(-2, 2)
range_frame = Window.partitionBy("region").orderBy("amount").rangeBetween(-200, 200)

rows_result = sales_df.withColumn("rows_neighbors", F.collect_list("amount").over(rows_frame))
range_result = sales_df.withColumn("range_neighbors", F.collect_list("amount").over(range_frame))

print("ROWS frame (physical neighbors):")
rows_result.select("region", "amount", "rows_neighbors").show(3, truncate=False)

print("RANGE frame (logical range):")
range_result.select("region", "amount", "range_neighbors").show(3, truncate=False)

# Mistake 3: Not handling nulls in ORDER BY
print("\n❌ Mistake: Nulls in ORDER BY column")
null_data = [("Alice", 100), ("Bob", None), ("Charlie", 200)]
null_df = spark.createDataFrame(null_data, ["name", "score"])

null_rank = null_df.withColumn("rank", rank().over(Window.orderBy("score")))
print("Rank with nulls (nulls first by default):")
null_rank.show()

print("\n✅ Solution: Handle nulls explicitly")
null_handled = null_df.withColumn("rank", rank().over(
    Window.orderBy(F.coalesce("score", -999))  # Nulls become -999
))
null_handled.show()

## 🎯 Key Takeaways

### What You Learned:
- ✅ **`Window.partitionBy().orderBy()`** - Define window specifications
- ✅ **`row_number()`, `rank()`, `dense_rank()`** - Ranking functions
- ✅ **`lag()`, `lead()`** - Access previous/next rows
- ✅ **Running totals and moving averages** - Aggregate over windows
- ✅ **Frame specifications** - ROWS vs RANGE frames
- ✅ **Performance optimization** - Efficient window operations

### Window Function Types:
- 🔸 **Ranking**: `row_number()`, `rank()`, `dense_rank()`, `percent_rank()`
- 🔸 **Aggregate**: `sum()`, `avg()`, `min()`, `max()`, `count()` over windows
- 🔸 **Navigation**: `lag()`, `lead()`, `first()`, `last()`
- 🔸 **Distribution**: `cume_dist()`, `percentile_approx()`

### Frame Types:
- 🔸 **ROWS**: Physical row positions (`rowsBetween(-1, 1)`)
- 🔸 **RANGE**: Logical value ranges (`rangeBetween(-100, 100)`)
- 🔸 **UNBOUNDED**: All rows (`unboundedPreceding` to `unboundedFollowing`)

### Performance Best Practices:
- 🔸 **Filter before windowing** to reduce data volume
- 🔸 **Choose narrow partitions** for better parallelism
- 🔸 **Use appropriate frame sizes** (smaller = faster)
- 🔸 **Cache windowed results** if reused multiple times
- 🔸 **Monitor shuffle operations** in Spark UI

### Common Patterns:
- 🔸 `Window.partitionBy("group").orderBy("sort_col")` - Basic window spec
- 🔸 `.rowsBetween(Window.unboundedPreceding, Window.currentRow)` - Running totals
- 🔸 `.rowsBetween(-N, N)` - Moving averages
- 🔸 `lag("col", offset)` - Compare with previous rows
- 🔸 `rank().over(window)` - Top N analysis

---

## 🚀 Next Steps

Now that you master window functions, you're ready for:

1. **DataFrame Joins** - Combining multiple DataFrames
2. **Complex Data Types** - Arrays, maps, and structs
3. **Spark SQL Integration** - SQL interface for DataFrames
4. **Advanced Analytics** - Time series and predictive modeling

**Window functions enable sophisticated analytical queries!**

---

**🎉 Congratulations! You now wield the analytical power of window functions like a data science expert!**